In [1]:
# Project name setup
project_name = "SeamTaping"

Add paths to the project directory and the directory containing utils

In [2]:
import sys
import os

# Add the path to the project directory
utils_path = os.path.abspath('../..')
if utils_path not in sys.path:
    sys.path.append(utils_path)
# Add the path to the directory containing utils to sys.path
utils_path = os.path.abspath('../../utils')
if utils_path not in sys.path:
    sys.path.append(utils_path)
print(sys.path)

['c:\\Users\\User\\.conda\\envs\\Azure-Custom-Vision-Practice\\python312.zip', 'c:\\Users\\User\\.conda\\envs\\Azure-Custom-Vision-Practice\\DLLs', 'c:\\Users\\User\\.conda\\envs\\Azure-Custom-Vision-Practice\\Lib', 'c:\\Users\\User\\.conda\\envs\\Azure-Custom-Vision-Practice', '', 'c:\\Users\\User\\.conda\\envs\\Azure-Custom-Vision-Practice\\Lib\\site-packages', 'c:\\Users\\User\\.conda\\envs\\Azure-Custom-Vision-Practice\\Lib\\site-packages\\win32', 'c:\\Users\\User\\.conda\\envs\\Azure-Custom-Vision-Practice\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\User\\.conda\\envs\\Azure-Custom-Vision-Practice\\Lib\\site-packages\\Pythonwin', 'd:\\Work\\Crack-Detection', 'd:\\Work\\Crack-Detection\\utils']


### Data Loading

Let's start by loading the dataset.



In [3]:
import os
import pandas as pd

# Read the dataset
data = pd.read_excel(f'../../dataset/{project_name}_All.xlsx')

### Authenticate the client

Instantiate a training and prediction client with your endpoint and keys. 

In [4]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
print(os.getenv("TRAINING_KEY"), os.getenv("TRAINING_ENDPOINT"))

# Authentication
credentials = ApiKeyCredentials(in_headers={"Training-key": os.getenv("TRAINING_KEY")})
trainer = CustomVisionTrainingClient(endpoint=os.getenv("TRAINING_ENDPOINT"), credentials=credentials)

# Authentication for prediction
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": os.getenv("PREDICTION_KEY")})
predictor = CustomVisionPredictionClient(endpoint=os.getenv("PREDICTION_ENDPOINT"), credentials=prediction_credentials)


90dad624b6664556accbcfd69e2e170d https://crackdetection.cognitiveservices.azure.com/


### Create or get the project

In [5]:
# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Find project by name
print(f"Searching for project '{project_name}'...")
projects = trainer.get_projects()

project_id = None
for project in projects:
    if project.name == project_name:
        project_id = project.id
        break

if project_id:
    print(f"Project '{project_name}' found with ID: {project_id}")
else:
    print(f"No project found with the name '{project_name}'")
    # Create a new project
    print ("Creating project...")
    project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)


Searching for project 'SeamTaping'...
Project 'SeamTaping' found with ID: a4b82d6b-8bd1-45aa-9308-39d665767e81


### Data Ingestion and Transformation

Create a data pipeline to transform and ingest data according to Azure Custom Vision's requirements.

In [9]:
# Function to create or retrieve existing tags in Custom Vision
def create_or_get_tags(trainer, project_id, labels):
    tag_dict = {}

    try:
        existing_tags = trainer.get_tags(project_id)
        existing_tag_names = {tag.name: tag for tag in existing_tags} if existing_tags else {}
    except Exception as e:
        print(f"Failed to retrieve existing tags: {str(e)}")
        existing_tag_names = {}

    for label in labels:
        if label in existing_tag_names:
            tag_dict[label] = existing_tag_names[label]
            print(f"Existing tag '{label}' (id: {tag_dict[label].id})'")
        else:
            try:
                print(f"Creating tag '{label}'")
                tag = trainer.create_tag(project_id, label)
                tag_dict[label] = tag
                existing_tag_names[label] = tag  # Update existing tag names dictionary
            except Exception as e:
                # Handle the exception due to tag creation failure
                print(f"Failed to create tag '{label}': {str(e)}")

    return tag_dict

# Assuming 'data' contains your labels from which you want to create tags
labels = data['Label'].unique()
# Create or get existing tags in your project
tag_dict = create_or_get_tags(trainer, project.id, labels)

Creating tag 'Damaged'


In [10]:
from azure.cognitiveservices.vision.customvision.training.models import Region, ImageUrlCreateEntry, ImageUrlCreateBatch
from tqdm import tqdm
import json
from utils.transform_functions import polygon_to_bbox

# Convert polygons to bounding boxes
data['bbox'] = data['2D Image Points'].apply(json.loads).apply(polygon_to_bbox)

def inject_dataset_to_custom_vision(data, trainer, project_id):

    for index, row in tqdm(data.iterrows(), total=data.shape[0]):
        img_width = 1000
        img_height = 750
        bbox = row['bbox']
        label = row['Label']

        regions = [Region(
            tag_id=tag_dict[label].id,
            left=bbox["left"] / img_width,
            top=bbox["top"] / img_height,
            width=bbox["width"] / img_width,
            height=bbox["height"] / img_height
        )]

        image_entry = ImageUrlCreateEntry(
            url=row['2D Image URL'],
            regions=regions
        )

        upload_result = trainer.create_images_from_urls(
            project_id, 
            ImageUrlCreateBatch(images=[image_entry])
            )

        # # If upload is not successful, print details about that image for debugging
        # if not upload_result.is_batch_successful:
        #     print("Image upload failed.")
        #     for image in upload_result.images:
        #         print(f"Image status: {image.status}, Source URL: {image.source_url}")
        #     break
        # else:
        #     print(f"Uploaded image: {row['2D Image URL']}")


inject_dataset_to_custom_vision(data, trainer, project.id)


100%|██████████| 328/328 [07:50<00:00,  1.44s/it]
